In [ ]:
using Plots

https://ru.wikipedia.org/wiki/Водородоподобный_атом

https://ru.wikipedia.org/wiki/Атом_водорода

In [ ]:
const nm = 1e-9 # m
const fm = 1e-15 # m
const ps = 1e-12 # s
const fs = 1e-15 # s

const c = 299792458 # m/s
const ħ = 1.0546e-34 # J*s
const e = 1.6022e-19 # Kl
const mₑ = 9.1094e-31 # kg
const mp = 1.6726e-27
const ε₀ = 8.8542e-12 # F/m

const k = 1/(4π*ε₀) # 8.987552e9 # N*m^2/q^2
const rₑ = k*e^2 / (mₑ*c^2) # 2.81794e-15 # m
const a₀ = 0.052917720859nm # 4π*ε₀*ħ^2 / (mₑ*q^2)
const α = 1/137.036 # e^2 / (4π*ε₀*ħ*c)
const vₑ = α*c # 2187 km/c

\begin{cases}r={\sqrt {x^{2}+y^{2}+z^{2}}},\\\theta =\arccos {\dfrac {z}{\sqrt {x^{2}+y^{2}+z^{2}}}}=\mathrm {arctg} {\dfrac {\sqrt {x^{2}+y^{2}}}{z}},\\\varphi =\mathrm {arctg} {\dfrac {y}{x}}.\end{cases}


$$ \psi _{nlm}(r,\theta ,\varphi )={\sqrt {\frac {(n-l-1)!}{2n{\cdot }(n+l)!}}}{\cdot }{\left({\frac {2}{na_{0}}}\right)}^{\frac {3}{2}}{\cdot }\exp {\left({-{\frac {r}{na_{0}}}}\right)}{\cdot }{\left({\frac {2r}{na_{0}}}\right)}^{l}L_{n-l-1}^{2l+1}{\left({\frac {2r}{na_{0}}}\right)}\cdot Y_{l,m}(\theta ,\varphi )$$

$$ L_{n}^{(\alpha )}(x)=\sum _{i=0}^{n}(-1)^{i}{n+\alpha \choose n-i}{\frac {x^{i}}{i!}}$$

$$ Y_{\ell }^{m}(\theta ,\varphi )=(-1)^{m}{\sqrt {{(2\ell +1) \over 4\pi }{(\ell -m)! \over (\ell +m)!}}}\,P_{\ell m}(\cos {\theta })\,e^{im\varphi }$$

$$P_{l}^{m}(x)=2^{l}\cdot (1-x^{2})^{m/2}\cdot \sum _{k=m}^{l}{\frac {k!}{(k-m)!}}\cdot x^{k-m}\cdot {\binom {l}{k}}{\binom {\frac {l+k-1}{2}}{l}}$$


$$ {\binom {\alpha }{k}}={\frac {\alpha ^{\underline {k}}}{k!}}={\frac {\alpha (\alpha -1)(\alpha -2)\cdots (\alpha -k+1)}{k(k-1)(k-2)\cdots 1}}\quad {\text{for }}k\in \mathbb {N} {\text{ and arbitrary }}\alpha $$

In [ ]:
bino(a, k) = k>0 ? prod(i-> a-i, 0:k-1) / factorial(k) : k<0 ? 0.0 : 1.0

P(x::Float64, m::Int64, l::Int64) = (-1)^m * 2^l * sqrt(1-x^2)^m * 
        sum( k-> factorial(k)/factorial(k-m) * x^(k-m) * binomial(l, k)*bino( 0.5(l+k-1), l ), m:l )

Y(θ::Float64, ϕ::Float64, m::Int64, l::Int64) = 
        sqrt( (2l+1)/(4π) * factorial(l-m)/factorial(l+m) ) * P( cos(θ), m, l ) * exp(im*m*ϕ)

L(x::Float64, a::Int64, n::Int64) = sum( i-> (-1)^i * binomial(n+a, n-i)*x^i / factorial(i), 0:n );

In [ ]:
function ψ(n, l, m, gr) # квантовые числа и радиус в борах
    
    a₀ = 1.0 # Bohr radius
    A = (2/(n*a₀))^2 * sqrt( 0.25a₀*factorial(n-l-1) / factorial(n+l)^3 )
    
    F(x) = x^l * exp(-0.5x) * factorial(n+l) * L( x, 2l+1, n-l-1 )
    R(r, θ, ϕ) = F( 2r/(n*a₀) ) * abs( Y(θ, ϕ, m, l) )
    R(x, z) = begin r = hypot(x,1,z); R( r, acos(z/r), atan(x,0) ) end
    
    X = range(-gr, stop = gr, length = 200)
    
    Sol = [ abs2( A*R(x, z) ) for x in X, z in X ]
    
    Sol /= (sum(Sol)*step(X)) # нормировка
    #println(sum(Sol)*step(X))
    
    return X, Sol
end;

In [ ]:
X, Z = ψ(4, 3, 0, 40);
heatmap(X, X, Z, xaxis="a₀", yaxis="a₀")

In [ ]:
Ps = Plots.Plot{Plots.GRBackend}[] # пустой массив специально под картинки

for n in 1:4, l in 0:n-1, m in 0:l # перебираем квантовые числа
    X, Z = ψ(n, l, m, 5n)
    p0 = heatmap(X, X, Z, title = "$n $l $m",
        legend = false, ticks = nothing, border = :none)
    push!(Ps, p0)
end;

In [ ]:
plot(Ps..., size = (600, 400))

In [ ]:
savefig("Sorb.png")

In [ ]:
X = collect(-3:0.01:3)
ψ100(r) = 4π*r*exp(-r)/sqrt(π)
ρᵣ = [ ψ100(x^2+y^2) for x in X, y in X ] 
ρᵣ /= maximum(ρᵣ)

heatmap(X, X, ρᵣ, xaxis="a₀", yaxis="a₀")

Θs = collect((0:0.01:1)*2π)
rp = 0.841fm/a₀ 
plot!( cos.(Θs), sin.(Θs), line = (2, :dash, :green), lab = "Bor's orbit" ) 
plot!( rp*cos.(Θs), rp*sin.(Θs), line = (2, :dash, :red), lab = "Proton radius" ) 
# , ticks = nothing, border = :none,

In [ ]:
n = 1
xx = 0
ξ = 1e-5
while xx < 3
    n+=1
    xx += (ξ*n)^2
end
n

In [ ]:
steps = [ (ξ*i)^2 for i in 1:n ]
Xadapt = [0.0; accumulate(+, steps ) ]

In [ ]:
ψ100(4e-5^2)

In [ ]:
plot(Xadapt, [ ψ100(x^2) for x in Xadapt ], line = 3, lab = "Radial distr")
vline!([rp 1], lab = ["Proton radius" "Bor's orbit"], line = (2, :dash) )
#lens!([0, 4e-5], [0, 6e-7], inset = (1, bbox(0.6, 0.3, 0.3, 0.3)), 
#        xticks = 0:1e-5:2e-5, yticks = 0:1e-7:4e-7,  subplot = 2 ) 

In [ ]:
xx = collect(0:1e-6:4e-5)
plot(xx, [ ψ100(x^2) for x in xx ], line = 3, 
    legend = false, xticks = 0:2e-5:4e-5, xaxis = (0,4e-5) )
vline!([rp], lab = "Proton radius", line = (2, :dash), size = (220, 150))

https://habr.com/ru/post/539210/

### Уровни энергии

$$
E_n = -\frac 12 \mu c^2 \frac{\alpha^2}{n^2}
$$

$$
\mu = \frac{m_em_p}{m_e+m_p}
$$

In [ ]:
M = mₑ*mp / (mₑ+mp)

In [ ]:
E(n) = -0.5M*c^2*α^2 / n^2 / e # eV = 6.242e+18 * J # eV = J/qₑ

!["series"](images/img_spec.png)

In [ ]:
E.(1:6)

In [ ]:
Eb = 12398 ./ ( E.(3:6) .- E(2) ) # Balmer series

In [ ]:
Ep = 12398 ./ ( E.(9:16) .- E(3) ) # Paschen series

In [ ]:
El = 12398 ./ ( E.(2:6) .- E(1) ) # Lyman series

In [ ]:
Ens = [Ep; Eb[1:2]]

In [ ]:
vline(Ens, xlim = (3500,9200) )

Данные оттудова:  
https://www.eso.org/sci/observing/tools/standards/IR_spectral_library.html

https://www.aanda.org/articles/aa/full_html/2019/09/aa36178-19/aa36178-19.html

http://cdsarc.u-strasbg.fr/viz-bin/cat/J/A+A/629/A100

In [ ]:
using DelimitedFiles

http://simbad.u-strasbg.fr/simbad/sim-id?Ident=HD102212

In [ ]:
names = readdir("data")
names = "data/" .* names[occursin.("J_A", names )]

In [ ]:
print( read(names[1], String)[1:209] )

In [ ]:
dats = readdlm(names[3], skipstart = 3);
size(dats)

In [ ]:
function velos(X, Y, E)
    dl = X[2] - X[1]
    function V(λ_rest)
        ix = findfirst(x-> abs(x-λ_rest)<dl, X)
        mn = minimum( Y[ix-3:ix+3] )
        ix = findfirst(x-> x==mn, Y)
        λ_obs = X[ix]
        (λ_obs-λ_rest) / λ_rest * c # m/c
    end
    [ V(e) for e in E ] * 1e-3 # km/s
end;

In [ ]:
plot(0.1dats[:, 1], dats[:, 2], legend = false, title = "HD196892")
vline!(0.1Ens, line = (1, :dash, :black, 0.9), xlab = "λ, nm" )
#vline!([380, 780]) # visible
#xaxis!((470,700))
lens!( [840, 890], [5e-13, 1e-12], inset = (1, bbox(0.45, 0.0, 0.3, 0.3)), yticks = nothing )

In [ ]:
plot(0.1dats[:, 1], dats[:, 2], legend = false, title = "HD196892", 
    background_color = RGB(0.2, 0.2, 0.2) )
vline!(0.1Ens, line = (1, :dash, :yellow, 0.9), xlab = "λ, nm" )
lens!( [840, 890], [5e-13, 1e-12], inset = (1, bbox(0.45, 0.0, 0.3, 0.3)), yticks = nothing )

In [ ]:
savefig("spector_dark.png")

In [ ]:
Vs = velos(dats[:,1], dats[:,2], Ens[1:end-1])

In [ ]:
sum(Vs) / length(Vs) # km/s

In [ ]:
# V(km/s) -34.565 [0.0012] 

http://simbad.cds.unistra.fr/simbad/sim-basic?Ident=HD196892&submit=SIMBAD+search

Doppler shift = (observed wavelength – rest wavelength) / rest wavelength.

With the Doppler shift being caused by an object’s motion it is no surprise that there is an equation linking the two. For electromagnetic radiation such as light the formula is:

Velocity = Doppler shift * speed of light (roughly 300,000 km/sec).

Perhaps a worked example will be useful.

Hydrogen, the most common element in the universe has amongst others an emission at 656.3 nanometres (nm). If we observe a certain galaxy, we might find this has shifted to 660.0nm, an amount of 3.7nm. Therefore, the Doppler shift is 3.7/656.3 = 0.006. Its recessional velocity is then approximately 0.006 * 300,000 = 1800 km/sec away from us.